In [1]:
from rating import get_rating_df

rating_df = get_rating_df()

In [2]:
rating_df[(rating_df['side'] == 'def') & (rating_df['role'] == 'qb1')]\
    .sort_values(by=['rating'], ascending=False)

,side,role,rating,season,week,team
38796,def,qb1,400.838710,2018,1,BAL
20244,def,qb1,347.919355,2021,2,SD
32760,def,qb1,344.806452,2019,1,ATL
20424,def,qb1,294.838710,2021,1,SD
13992,def,qb1,278.032258,2022,1,SF
...,...,...,...,...,...,...
38844,def,qb1,-197.064516,2018,1,NO
26460,def,qb1,-197.483871,2020,2,SEA
38832,def,qb1,-238.838710,2018,1,TB
38484,def,qb1,-239.419355,2018,2,TB


In [9]:
import pandas as pd
from stats import stats_roles, get_role_week

def _compare_teams(team1_df, team2_df):
    off_df = team1_df[team1_df['side'] == 'off'].reset_index().set_index('role')
    def_df = team2_df[team2_df['side'] == 'def'].reset_index().set_index('role')
    
    return {
        role : off_df.loc[role]['rating'] - def_df.loc[role]['rating'] for role in stats_roles
    }

alias = {
    'NYG': 'NYG',
    'DAL': 'DAL',
    'NO': 'NO',
    'ATL': 'ATL',
    'LAR': 'STL',
    'CHI': 'CHI',
    'MIN': 'MIN',
    'GB': 'GB',
    'PIT': 'PIT',
    'IND': 'IND',
    'DEN': 'DEN',
    'NYJ': 'NYJ',
    'PHI': 'PHI',
    'TB': 'TB',
    'CIN': 'CIN',
    'CAR': 'CAR',
    'JAX': 'JAX',
    'HOU': 'HOU',
    'WSH': 'WSH',
    'ARI': 'ARI',
    'NE': 'NE',
    'SF': 'SF',
    'CLE': 'CLE',
    'LV': 'OAK',
    'KC': 'KC',
    'LAC': 'SD',
    'BUF': 'BUF',
    'BAL': 'BAL',
    'TEN': 'TEN',
    'MIA': 'MIA',
    'SEA': 'SEA',
    'DET': 'DET'
}

def compare_teams(season, week, team1, team2, show_vs=False):
    rating_df = get_rating_df()
    
    team1_alias = alias[team1] if team1 in alias else team1
    team2_alias = alias[team2] if team2 in alias else team2

    week_df = rating_df[(rating_df['season'] == season) & (rating_df['week'] == week)]
    
    team1_df = week_df[week_df['team'] == team1_alias]
    if len(team1_df.index) == 0:
        print('==========')
        print(rating_df[(rating_df['season'] == season)])
        print('----------')
        print(week_df['team'])
        raise Exception(f"Can not find: {team1} > {team1_alias}")
    team2_df = week_df[week_df['team'] == team2_alias]
    if len(team2_df.index) == 0:
        raise Exception(f"Can not find: {team2} > {team2_alias}")
    
    team1_res = _compare_teams(team1_df, team2_df)
    team1_res['rating'] = sum(team1_res.values()) 
    team1_res['team'] = team1
    if show_vs:
        team1_res['vs'] = team2
    
    team2_res = _compare_teams(team2_df, team1_df)
    team2_res['rating'] = sum(team2_res.values()) 
    team2_res['team'] = team2
    if show_vs:
        team2_res['vs'] = team1
    
    return pd.DataFrame([team1_res, team2_res])

schedule = [
    {
        'home': 'NYG',
        'away': 'DAL'
    },{
        'home': 'NO',
        'away': 'ATL'
    },{
        'home': 'LAR',
        'away': 'CHI'
    },{
        'home': 'MIN',
        'away': 'GB'
    },{
        'home': 'PIT',
        'away': 'IND'
    },{
        'home': 'DEN',
        'away': 'NYJ'
    },{
        'home': 'PHI',
        'away': 'TB'
    },{
        'home': 'CIN',
        'away': 'CAR'
    },{
        'home': 'JAX',
        'away': 'HOU'
    },{
        'home': 'WSH',
        'away': 'ARI'
    },{
        'home': 'NE',
        'away': 'SF'
    },{
        'home': 'CLE',
        'away': 'LV'
    },{
        'home': 'KC',
        'away': 'LAC'
    },{
        'home': 'BUF',
        'away': 'BAL'
    },{
        'home': 'TEN',
        'away': 'MIA'
    },{
        'home': 'SEA',
        'away': 'DET'
    }
]

def compare_week(season, week):
    return pd.concat([
        compare_teams(season, week, game['home'], game['away']) for game in schedule
    ])

comparisions = compare_week(2024, 3)

def sort_and_return(df, role):
    rtn = df.sort_values(by=[role], ascending=False).head(3)
    
    rtn.style.format(precision=2)

    return rtn.to_markdown()

print('--qb1--')
print(sort_and_return(comparisions, 'qb1'))

print('--rb1--')
print(sort_and_return(comparisions, 'rb1'))

print('--rb2--')
print(sort_and_return(comparisions, 'rb2'))

print('--wr1--')
print(sort_and_return(comparisions, 'wr1'))

print('--wr2--')
print(sort_and_return(comparisions, 'wr2'))

print('--wr3--')
print(sort_and_return(comparisions, 'wr3'))

--qb1--
|    |      wr1 |     wr2 |     wr3 |     rb1 |      rb2 |     qb1 |    rank | team   |
|---:|---------:|--------:|--------:|--------:|---------:|--------:|--------:|:-------|
|  0 |  85.7447 | 82.2868 | 65.8441 | 77.8833 |  23.8728 | 228.779 | 564.41  | PIT    |
|  0 |  84.9585 | 85.6637 | 30.5622 | 27.9117 | -17.9709 | 214.847 | 425.973 | BUF    |
|  1 | 131.432  | 23.7611 | 56.6405 | 52.0055 |  47.4079 | 193.135 | 504.382 | ARI    |
--rb1--
|    |       wr1 |     wr2 |      wr3 |     rb1 |      rb2 |        qb1 |     rank | team   |
|---:|----------:|--------:|---------:|--------:|---------:|-----------:|---------:|:-------|
|  1 | -22.5542  | 98.8507 | -6.24731 | 85.6684 | -90.9032 |   6.65143  |  71.4658 | BAL    |
|  0 |  85.7447  | 82.2868 | 65.8441  | 77.8833 |  23.8728 | 228.779    | 564.41   | PIT    |
|  0 |   6.50887 | 49.8581 |  6.45084 | 77.1385 | -15.797  |  -0.143714 | 124.016  | NYG    |
--rb2--
|    |      wr1 |      wr2 |     wr3 |      rb1 |     rb2 |      q

In [10]:
from stats import get_game_df

def run_week(season, week):
    games_df = get_game_df()

    week_df = games_df[
        (games_df['season'] == season) & 
        (games_df['week'] == week)
    ]

    for i, row in week_df[['homeTeamDisplay', 'homeScore', 'awayTeamDisplay', 'awayScore']].iterrows():
        print(f"{row['homeTeamDisplay']} ({row['homeScore']}) v {row['awayTeamDisplay']} ({row['awayScore']})")
        print(compare_teams(season, week, row['homeTeamDisplay'], row['awayTeamDisplay']).to_markdown())

run_week(2024, 2)

BAL (23) v OAK (26)
|    |      wr1 |      wr2 |     wr3 |      rb1 |       rb2 |       qb1 |    rank | team   |
|---:|---------:|---------:|--------:|---------:|----------:|----------:|--------:|:-------|
|  0 |  65.1328 | -36.1647 | 59.4332 |  43.8871 | 122.161   |   3.80197 | 258.252 | BAL    |
|  1 | 101.136  |  65.8309 | 45.2005 | -14.795  |   5.28495 | 153.069   | 355.727 | OAK    |
JAX (13) v CLE (18)
|    |     wr1 |     wr2 |     wr3 |      rb1 |      rb2 |      qb1 |    rank | team   |
|---:|--------:|--------:|--------:|---------:|---------:|---------:|--------:|:-------|
|  0 | 85.2419 | 48.0484 | 94.2796 |  3.93871 | -10.5484 |  84.5218 | 305.482 | JAX    |
|  1 | 55.7759 | 84.1367 | 50.7068 | 30.7581  |  24.1649 | 146.291  | 391.834 | CLE    |
TEN (17) v NYJ (24)
|    |     wr1 |      wr2 |      wr3 |      rb1 |      rb2 |     qb1 |     rank | team   |
|---:|--------:|---------:|---------:|---------:|---------:|--------:|---------:|:-------|
|  0 | 80.6886 | 21.1419  |  4

In [7]:
get_role_week(2024, 3, 'ATL', 'wr2')

season                              2024
week                                   3
gameId                         401671793
gameDisplay                    KC vs ATL
gameDate               2024-09-23T00:20Z
teamId                                 1
teamDisplay                          ATL
playerId                         4040655
playerDisplay             Darnell Mooney
playerPosition                        wr
playerPositionGroup             catching
playerPositionNorm                    wr
passCmp                                0
passAtt                                0
passYds                                0
passTd                                 0
passInt                                0
passLong                               0
passRating                             0
passTargetYds                          0
rushAtt                                0
rushYds                                0
rushTd                                 0
rushLong                               0
rushYdsBc       